In [1]:
import numpy as np
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from time import time
import matplotlib.pyplot as plt
import os
import pickle
import scipy
from sklearn.preprocessing import StandardScaler
import pandas as pd
from scipy.special import comb

In [2]:
features_folder = "C:\\Users\\t-anmend\\Documents\\train_1\\"

In [3]:
with open(os.path.join(features_folder,'train1_stats_features.txt'),'rb') as f:
    features  = pickle.load(f)


In [4]:
features[0]

[array([ 1.56789474e+02,  2.24210526e+01,  1.66157895e+02,  7.66842105e+01,
         2.30894737e+02,  7.04210526e+01,  9.59473684e+01,  9.49473684e+01,
         1.58842105e+02,  1.84947368e+02,  1.13210526e+02,  1.03526316e+02,
         1.26736842e+02,  1.82052632e+02,  1.25368421e+02,  1.16473684e+02,
         1.13736842e+02,  1.54789474e+02,  1.40473684e+02,  1.07473684e+02,
         4.00526316e+01,  2.06894737e+02,  9.52105263e+01,  9.36315789e+01,
         1.20368421e+02,  1.58315789e+02,  1.91684211e+02,  2.04842105e+02,
         1.40210526e+02,  1.01368421e+02,  1.52000000e+02,  1.21315789e+02,
         1.80842105e+02,  6.92105263e+01,  5.88421053e+01,  1.65315789e+02,
         1.08842105e+02,  9.95263158e+01,  1.28105263e+02,  7.66315789e+01,
         1.07263158e+02,  1.39578947e+02,  4.34210526e+01,  1.32421053e+02,
         8.69473684e+01,  1.64894737e+02,  1.33631579e+02,  7.93157895e+01,
         1.07368421e+02,  2.21947368e+02,  1.85368421e+02,  8.40000000e+01,
         1.1

In [5]:
features = np.array(features)

In [97]:
features.shape

(3000, 3)

In [7]:
features_stats = np.zeros((features.shape[0],len(features[0][0])))
for i in range(features.shape[0]):
    features_stats[i] = features[i][0]

In [98]:
features_stats.shape

(3000, 896)

In [52]:
labels = []
for i in range(features.shape[0]):
    labels.append(features[i][1])

In [10]:
len(labels)

3000

In [11]:
labels

[['drilling', 'dog_bark', 'air_conditioner', 'children_playing', 'jackhammer'],
 ['dog_bark', 'drilling', 'siren', 'street_music', 'gun_shot', 'dog_bark'],
 ['drilling', 'children_playing', 'dog_bark', 'street_music'],
 ['air_conditioner',
  'air_conditioner',
  'siren',
  'jackhammer',
  'engine_idling'],
 ['jackhammer',
  'jackhammer',
  'air_conditioner',
  'children_playing',
  'gun_shot',
  'drilling',
  'gun_shot',
  'dog_bark',
  'dog_bark'],
 ['air_conditioner', 'air_conditioner', 'drilling', 'jackhammer', 'gun_shot'],
 ['children_playing',
  'air_conditioner',
  'siren',
  'engine_idling',
  'gun_shot',
  'street_music'],
 ['car_horn',
  'dog_bark',
  'gun_shot',
  'siren',
  'children_playing',
  'engine_idling',
  'siren'],
 ['car_horn',
  'engine_idling',
  'drilling',
  'dog_bark',
  'gun_shot',
  'drilling',
  'siren',
  'children_playing'],
 ['gun_shot',
  'drilling',
  'dog_bark',
  'car_horn',
  'children_playing',
  'jackhammer',
  'street_music',
  'jackhammer'],
 ['

In [12]:
n_samples, n_features = features_stats.shape
labels_ = []

for i in range(len(labels)):
    for j in range(len(labels[i])):
        labels_.append(labels[i][j])
    
n_labels = len(np.unique(labels_))

sample_size = 300

In [13]:
print("n_labels: %d, \t n_samples %d, \t n_features %d"
      % (n_labels, n_samples, n_features))

n_labels: 10, 	 n_samples 3000, 	 n_features 896


In [14]:
print(82 * '_')
print('init\t\ttime\tinertia\t\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')

def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('%-9s\t%.2fs\t%i\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_,
                                                average_method='arithmetic'),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=sample_size)))

for i in range(8,21):    
    bench_k_means(KMeans(init='k-means++', n_clusters=i, n_init=10),
                  name=f"k-means++ {i}", data=features_stats)

    bench_k_means(KMeans(init='random', n_clusters=i, n_init=10),
                  name=f"random {i}", data=features_stats)

    # in this case the seeding of the centers is deterministic, hence we run the
    # kmeans algorithm only once with n_init=1
    pca = PCA(n_components=i).fit(features_stats)
    bench_k_means(KMeans(init=pca.components_, n_clusters=i, n_init=1),
                  name=f"PCA-based {i}",
                  data=features_stats)
print(82 * '_')

__________________________________________________________________________________
init		time	inertia		homo	compl	v-meas	ARI	AMI	silhouette
k-means++ 8	7.30s	1286676860	0.253	0.986	0.403	0.000	0.002	0.034
random 8 	6.94s	1286978073	0.253	0.986	0.403	0.000	0.002	0.035
PCA-based 8	0.65s	1301553101	0.237	0.987	0.382	0.000	0.003	0.040
k-means++ 9	7.41s	1275317441	0.269	0.986	0.422	0.000	0.002	0.033
random 9 	6.41s	1275295992	0.267	0.986	0.420	0.000	0.002	0.033
PCA-based 9	0.66s	1287924616	0.252	0.987	0.402	0.000	0.003	0.035
k-means++ 10	7.59s	1264890490	0.282	0.987	0.438	0.000	0.002	0.031
random 10	6.37s	1265158232	0.281	0.986	0.438	0.000	0.001	0.032
PCA-based 10	0.65s	1287422827	0.253	0.987	0.402	0.000	0.003	0.035
k-means++ 11	7.62s	1254742151	0.295	0.988	0.454	0.000	0.003	0.031
random 11	6.73s	1255142204	0.295	0.987	0.454	0.000	0.002	0.031
PCA-based 11	0.84s	1275968006	0.270	0.988	0.424	0.000	0.003	0.032
k-means++ 12	8.51s	1247843685	0.306	0.987	0.467	0.000	0.002	0.025
random 12	6.92s	12

In [15]:
pca = PCA(n_components=n_labels).fit(features_stats)

In [16]:
estimator_PCA = KMeans(init=pca.components_, n_clusters=n_labels, n_init=1)

In [81]:
predicted_labels_PCA = estimator_PCA.fit_predict(features_stats)

In [82]:
predicted_labels_PCA.shape

(3000,)

In [17]:
cluster_matrix_pca = metrics.cluster.contingency_matrix(labels,estimator_PCA.fit_predict(features_stats))

In [41]:
cluster_matrix_pca

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [19]:
estimator = KMeans(init='k-means++', n_clusters=10, n_init=10)

In [36]:
predicted_clusters = estimator.fit_predict(features_stats)

In [37]:
cluster_matrix = metrics.cluster.contingency_matrix(labels,predicted_clusters)

In [39]:
predicted_clusters

array([9, 9, 9, ..., 0, 5, 7])

In [40]:
predicted_clusters.shape

(3000,)

In [38]:
cluster_matrix

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [23]:
cluster_matrix.shape

(2915, 10)

In [25]:
labels

[['drilling', 'dog_bark', 'air_conditioner', 'children_playing', 'jackhammer'],
 ['dog_bark', 'drilling', 'siren', 'street_music', 'gun_shot', 'dog_bark'],
 ['drilling', 'children_playing', 'dog_bark', 'street_music'],
 ['air_conditioner',
  'air_conditioner',
  'siren',
  'jackhammer',
  'engine_idling'],
 ['jackhammer',
  'jackhammer',
  'air_conditioner',
  'children_playing',
  'gun_shot',
  'drilling',
  'gun_shot',
  'dog_bark',
  'dog_bark'],
 ['air_conditioner', 'air_conditioner', 'drilling', 'jackhammer', 'gun_shot'],
 ['children_playing',
  'air_conditioner',
  'siren',
  'engine_idling',
  'gun_shot',
  'street_music'],
 ['car_horn',
  'dog_bark',
  'gun_shot',
  'siren',
  'children_playing',
  'engine_idling',
  'siren'],
 ['car_horn',
  'engine_idling',
  'drilling',
  'dog_bark',
  'gun_shot',
  'drilling',
  'siren',
  'children_playing'],
 ['gun_shot',
  'drilling',
  'dog_bark',
  'car_horn',
  'children_playing',
  'jackhammer',
  'street_music',
  'jackhammer'],
 ['

In [47]:
data = pd.DataFrame(columns=('recording_index', 'label', 'assigned_cluster'))

In [48]:
data

,recording_index,label,assigned_cluster


In [32]:
data

,recording_index,label,assigned_cluster
0,0,go,1


In [62]:
start = 0
for i in range(len(labels)):
    for label_ in range(len(labels[i])):
        
        data.loc[start] = [i] + [labels[i][label_]] + [predicted_clusters[i]]
        start = start + 1

In [63]:
data.tail()

,recording_index,label,assigned_cluster
18105,2998,dog_bark,5
18106,2999,air_conditioner,7
18107,2999,air_conditioner,7
18108,2999,engine_idling,7
18109,2999,gun_shot,7


In [64]:
labels[2999]

['air_conditioner', 'air_conditioner', 'engine_idling', 'gun_shot']

In [65]:
cluster_matrix = metrics.cluster.contingency_matrix(data['label'],data['assigned_cluster'])

In [66]:
cluster_matrix

array([[148, 262, 173, 260, 182, 156,  99, 290,  87, 129],
       [109, 384, 195, 240, 267, 120,  99, 138, 101,  93],
       [ 90, 280, 174, 358, 482,  95, 112, 122,  58, 101],
       [ 90, 228, 174, 256, 229, 108, 354, 163,  89, 130],
       [154, 348, 168, 225, 210, 295,  93, 158, 135,  99],
       [267, 269, 167, 163, 211, 151, 115, 278,  87,  83],
       [122, 301, 164, 253, 210, 112, 128, 156, 239,  88],
       [235, 348, 180, 166, 195, 204,  72, 209, 122,  85],
       [ 88, 260, 493, 148, 321, 105, 132, 177,  44, 113],
       [105, 367, 139, 296, 359, 103, 109,  96,  82,  83]])

In [83]:
data_PCA = pd.DataFrame(columns=('recording_index', 'label', 'assigned_cluster'))

In [84]:
data_PCA

,recording_index,label,assigned_cluster


In [32]:
data

,recording_index,label,assigned_cluster
0,0,go,1


In [87]:
start = 0
for i in range(len(labels)):
    for label_ in range(len(labels[i])):
        
        data_PCA.loc[start] = [i] + [labels[i][label_]] + [predicted_labels_PCA[i]]
        start = start + 1

In [88]:
data_PCA.tail()

,recording_index,label,assigned_cluster
18105,2998,dog_bark,1
18106,2999,air_conditioner,5
18107,2999,air_conditioner,5
18108,2999,engine_idling,5
18109,2999,gun_shot,5


In [64]:
labels[2999]

['air_conditioner', 'air_conditioner', 'engine_idling', 'gun_shot']

In [89]:
cluster_matrix_PCA = metrics.cluster.contingency_matrix(data_PCA['label'],data_PCAestimator_PCA.fit_predict(features_stats))['assigned_cluster'])

In [90]:
cluster_matrix_PCA

array([[264, 150, 166,   0, 365, 279, 120, 239, 203,   0],
       [450, 129, 193,   1, 216, 229, 110, 280, 138,   0],
       [346,  97, 172,   0, 233, 210, 140, 573, 100,   1],
       [290, 114, 164,   1, 242, 240, 389, 262, 118,   1],
       [349, 325, 165,   0, 247, 255, 112, 243, 189,   0],
       [295, 136, 153,   0, 310, 207, 122, 217, 348,   3],
       [432, 129, 167,   1, 221, 300, 144, 224, 154,   1],
       [364, 203, 177,   1, 279, 206,  84, 208, 294,   0],
       [260,  97, 497,   0, 278, 175, 146, 312, 114,   2],
       [456, 116, 136,   0, 174, 202, 125, 406, 123,   1]])

In [67]:
def get_true_positives(matrix):

    TP = 0
    for i in range(matrix.shape[1]):
        for j in range(matrix.shape[0]):
            TP = TP + comb(matrix[j][i], 2, exact=True)
            
    return TP

In [68]:
def get_false_positives(matrix):

    TP_FP = 0
    sum_array = np.sum(matrix, axis=0)
    for i in range(sum_array.shape[0]):
        TP_FP = TP_FP + comb(sum_array[i], 2, exact=True)
        
    TP = 0
    for i in range(matrix.shape[1]):
        for j in range(matrix.shape[0]):
            TP = TP + comb(matrix[j][i], 2, exact=True)
        
    return TP_FP - TP

In [69]:
def get_false_negatives(matrix):    
    FN = 0

    labels = matrix.shape[0]

    clusters = matrix.shape[1]

    for label in range(labels):
        for cluster_ in range(clusters):
            for cluster in range(cluster_ + 1,clusters):

                FN = FN + matrix[label][cluster_] * matrix[label][cluster]
                
    return FN

In [70]:
def get_precision(tp, fp):

    return tp / (tp + fp)

In [71]:
def get_recall(tp, fn):
    
    return tp / (tp + fn)

In [72]:
def get_f_measure(precision, recall):
    
    return 2 * precision * recall / (precision + recall)

In [75]:
precision = get_precision(get_true_positives(cluster_matrix), get_false_positives(cluster_matrix))

In [76]:
precision

0.11123390644878359

In [77]:
recall = get_recall(get_true_positives(cluster_matrix), get_false_negatives(cluster_matrix))

In [78]:
recall

0.12604385890041822

In [79]:
f_measure = get_f_measure(precision, recall)

In [80]:
f_measure

0.11817669294667404

In [91]:
precision_PCA = get_precision(get_true_positives(cluster_matrix_PCA), get_false_positives(cluster_matrix_PCA))

In [92]:
precision_PCA

0.11036179175703606

In [93]:
recall_PCA = get_recall(get_true_positives(cluster_matrix_PCA), get_false_negatives(cluster_matrix_PCA))

In [94]:
recall_PCA

0.14983441649494592

In [95]:
f_measure_PCA = get_f_measure(precision_PCA, recall_PCA)

In [96]:
f_measure_PCA

0.12710404031128897